In [1]:
from core.config.config import get_config

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

Settings.llm = OpenAI(
    model="gpt-3.5-turbo", 
    temperature=0.0,
    api_key=os.getenv("api_key"),
    api_version=os.getenv("api_version"),
)

c:\ProgramData\miniconda3\envs\cb_uit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from llama_index.embeddings.huggingface_optimum import OptimumEmbedding
from pathlib import Path
from transformers import AutoTokenizer

embed_path = str(Path(
    get_config("Path", "model"),
    get_config("Model", "embed")
))

tokenizer = AutoTokenizer.from_pretrained(embed_path)
tokenizer.model_input_names = ["input_ids", "attention_mask"]

Settings.embed_model = OptimumEmbedding(
    folder_name=embed_path,
    tokenizer=tokenizer,
    pooling='mean',
    max_length=256
)

The ONNX file model_optimized_quantized.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.


In [5]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.retrievers.bm25 import BM25Retriever

import chromadb
from unidecode import unidecode
import joblib
from os import path

In [6]:
path_index = get_config("Path", "Index")

chroma_client = chromadb.PersistentClient(path_index)
nodes_dict = joblib.load(path.join(path_index, "nodes_dict.pkl"))

In [7]:

from llama_index.core.retrievers import RecursiveRetriever

def get_retriever(intent, top_vector=10, top_bm25=None, **kargs):
    """
     @brief Create a retriever for a given intent. It is used to retrieve documents that are similar to the intent's one or more documents.
     @param intent The intent to use for the retrieval.
     @param top_vector The number of documents to retrieve.
     @param top_bm25 The BM25 similarity to use.
     @return An instance of : class : ` skbio. core. retriever. Retriever `. Note that this will be a : class
    """
    clean_intent = unidecode(intent.lower())
    path_save = path.join(get_config("Path", "Index"), clean_intent)
    
    storage_context = StorageContext.from_defaults(
        vector_store=ChromaVectorStore(
            chroma_collection=chroma_client.get_or_create_collection(clean_intent)
        ),
        persist_dir=path_save,
    )
    
    index = VectorStoreIndex(
        nodes = [],
        storage_context = storage_context
    )

    # Set the top vector to the vector if not set yet.
    if not top_bm25:
        top_bm25 = top_vector

    vector_retriever = RecursiveRetriever(
        "root",
        retriever_dict={
            "root": index.as_retriever(similarity_top_k=top_vector, **kargs),
        },
        node_dict=nodes_dict[intent],
        verbose=True,
    )
    
    bm25_retriever = BM25Retriever.from_defaults(similarity_top_k=top_bm25, docstore=storage_context.docstore, verbose=True)
    
    return vector_retriever, bm25_retriever

In [8]:
from llama_index.core.retrievers import QueryFusionRetriever

intent = "hỏi_đáp_điểm_chuẩn"

vector_retriever, bm25_retriever = get_retriever(intent, top_vector=10)

retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=None,
    num_queries=1,  # set this to 1 to disable query generation
    use_async=False,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)

retriever.retrieve("Điểm chuẩn ngành Khoa học máy tính 2022")

Retrieving with query id None: Điểm chuẩn ngành Khoa học máy tính 2022


BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: f974aa51-71b3-45f2-9bc9-e90c75328f07
Retrieving with query id f974aa51-71b3-45f2-9bc9-e90c75328f07: Điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: 23cccb5a-47f0-4d0a-9f14-5698e728ebdc
Retrieving with query id 23cccb5a-47f0-4d0a-9f14-5698e728ebdc: Điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb
Retrieving with query id aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb: Điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: aacc196e-64ba-4092-a560-5b92a45e8eb6
Retrieving with query id aacc196e-64ba-4092-a560-5b92a45e8eb6: Điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: 93378231-34cf-480d-a2f8-1405e72ac165
Retrieving with query id 93378231-34cf-480d-a2f8-1405e72ac165: Điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: 0b3a1557-53d7-4a44-8793-ac0cfddf2dcc
Retrieving with query id 0b3a1557-53d7-4a44-8793-ac0cfddf2d

[NodeWithScore(node=TextNode(id_='f974aa51-71b3-45f2-9bc9-e90c75328f07', embedding=None, metadata={'pattern': 'hỏi_đáp_điểm_chuẩn|thpt|khmt|năm_2022'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['pattern'], relationships={}, text='Điểm chuẩn ngành Khoa học máy tính năm 2022 là 27.1', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=7.6672528050333275),
 NodeWithScore(node=TextNode(id_='aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb', embedding=None, metadata={'pattern': 'hỏi_đáp_điểm_chuẩn|dgnl|khmt|năm_2022'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['pattern'], relationships={}, text='Điểm chuẩn ĐGNL năm 2022 ngành Khoa học máy tính là 888', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=7.6672528050333275),
 NodeWithScore(node=TextNode(id_='93378231

In [9]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever)

In [47]:
from llama_index.core.chat_engine import ContextChatEngine, SimpleChatEngine
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=1000)

chat_engine = ContextChatEngine.from_defaults(
    retriever=retriever,
    context_template=(
        "You are an advising education enrollment chatbot for VNUHCM - University of Information Technology (UIT), able to have normal interactions."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
)

In [59]:
chat_engine.reset()
memory.reset()

chat_engine.chat("Bạn là ai?", chat_history=memory.get_all())

memory.set(chat_engine.chat_history)

Retrieving with query id None: Bạn là ai?
Retrieved node with id, entering: 5276e1ce-9f3a-4f7b-b8bc-6f13f8ddde4f
Retrieving with query id 5276e1ce-9f3a-4f7b-b8bc-6f13f8ddde4f: Bạn là ai?
Retrieved node with id, entering: 62169517-3883-4eab-a87e-02bf72999c4d
Retrieving with query id 62169517-3883-4eab-a87e-02bf72999c4d: Bạn là ai?
Retrieved node with id, entering: 87a8a037-317b-4c22-a3bf-b5d19bda6b37
Retrieving with query id 87a8a037-317b-4c22-a3bf-b5d19bda6b37: Bạn là ai?
Retrieved node with id, entering: b09ec4e1-330e-45e1-b5e6-26537be0a3aa
Retrieving with query id b09ec4e1-330e-45e1-b5e6-26537be0a3aa: Bạn là ai?
Retrieved node with id, entering: db25244b-a585-4d73-a6ff-735c4dac860a
Retrieving with query id db25244b-a585-4d73-a6ff-735c4dac860a: Bạn là ai?
Retrieved node with id, entering: 0b3a1557-53d7-4a44-8793-ac0cfddf2dcc
Retrieving with query id 0b3a1557-53d7-4a44-8793-ac0cfddf2dcc: Bạn là ai?
Retrieved node with id, entering: d5b8610d-f3f8-4de4-ac55-cc965a7bb6e0
Retrieving with q

BM25Retriever does not support embeddings, skipping...


In [60]:
chat_engine.reset()

chat_engine.chat("Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Khoa học máy tính 2022", chat_history=memory.get_all())

memory.set(chat_engine.chat_history)

Retrieving with query id None: Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Khoa học máy tính 2022


BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: f974aa51-71b3-45f2-9bc9-e90c75328f07
Retrieving with query id f974aa51-71b3-45f2-9bc9-e90c75328f07: Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: 23cccb5a-47f0-4d0a-9f14-5698e728ebdc
Retrieving with query id 23cccb5a-47f0-4d0a-9f14-5698e728ebdc: Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb
Retrieving with query id aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb: Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: aacc196e-64ba-4092-a560-5b92a45e8eb6
Retrieving with query id aacc196e-64ba-4092-a560-5b92a45e8eb6: Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Khoa học máy tính 2022
Retrieved node with id, entering: 93378231-34cf-480d-a2f8-1405e72ac165
Retrieving with query id 93378231-34cf-480d-a2f8-1405e72ac165: Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Kho

In [61]:
chat_engine.reset()
chat_engine.chat("Điểm chuẩn ngành Khoa học máy tính 2021", chat_history=memory.get_all())

memory.set(chat_engine.chat_history)

Retrieving with query id None: Điểm chuẩn ngành Khoa học máy tính 2021


BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb
Retrieving with query id aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb: Điểm chuẩn ngành Khoa học máy tính 2021
Retrieved node with id, entering: 93378231-34cf-480d-a2f8-1405e72ac165
Retrieving with query id 93378231-34cf-480d-a2f8-1405e72ac165: Điểm chuẩn ngành Khoa học máy tính 2021
Retrieved node with id, entering: 0b3a1557-53d7-4a44-8793-ac0cfddf2dcc
Retrieving with query id 0b3a1557-53d7-4a44-8793-ac0cfddf2dcc: Điểm chuẩn ngành Khoa học máy tính 2021
Retrieved node with id, entering: f974aa51-71b3-45f2-9bc9-e90c75328f07
Retrieving with query id f974aa51-71b3-45f2-9bc9-e90c75328f07: Điểm chuẩn ngành Khoa học máy tính 2021
Retrieved node with id, entering: 23cccb5a-47f0-4d0a-9f14-5698e728ebdc
Retrieving with query id 23cccb5a-47f0-4d0a-9f14-5698e728ebdc: Điểm chuẩn ngành Khoa học máy tính 2021
Retrieved node with id, entering: 466a4048-b647-464d-99d9-fbbef0152130
Retrieving with query id 466a4048-b647-464d-99d9-fbbef01521

In [62]:
chat_engine.reset()
chat_engine.chat("Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022", chat_history=memory.get_all())

memory.set(chat_engine.chat_history)

Retrieving with query id None: Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022


BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: 93378231-34cf-480d-a2f8-1405e72ac165
Retrieving with query id 93378231-34cf-480d-a2f8-1405e72ac165: Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022
Retrieved node with id, entering: aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb
Retrieving with query id aef20c66-bc37-4bd5-ae7f-b3d3ac15d6fb: Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022
Retrieved node with id, entering: f974aa51-71b3-45f2-9bc9-e90c75328f07
Retrieving with query id f974aa51-71b3-45f2-9bc9-e90c75328f07: Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022
Retrieved node with id, entering: 0b3a1557-53d7-4a44-8793-ac0cfddf2dcc
Retrieving with query id 0b3a1557-53d7-4a44-8793-ac0cfddf2dcc: Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022
Retrieved node with id, entering: da33359d-f272-4284-bdf7-78080601621e
Retrieving with query id da33359d-f272-4284-bdf7-78080601621e: Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022
Retrieved node with id, entering: 23cccb5a-47f0-4d0a-9f14-5698e728ebdc

In [64]:
for msg in memory.get_all():
    print(f"{msg.role:<15} : {msg.content}")
    print()

user            : Bạn là ai?

assistant       : Xin chào! Mình là chatbot hỗ trợ tư vấn về việc xét tuyển vào Trường Đại học Công nghệ Thông tin - ĐHQG TP.HCM (UIT). Bạn có thắc mắc gì cần tư vấn không?

user            : Tôi tên là Đại, tôi muốn biết điểm chuẩn ngành Khoa học máy tính 2022

assistant       : Điểm chuẩn ngành Khoa học máy tính năm 2022 là 27.1 điểm. Bạn cần thêm thông tin gì khác không?

user            : Điểm chuẩn ngành Khoa học máy tính 2021

assistant       : Điểm chuẩn ngành Khoa học máy tính năm 2021 là 27.3 điểm. Nếu cần thêm thông tin hoặc có câu hỏi nào khác, bạn hãy thoải mái hỏi nhé!

user            : Điểm chuẩn ngành Khoa học máy tính 2021 so với 2022

assistant       : Điểm chuẩn ngành Khoa học máy tính giữa năm 2021 và 2022 có sự thay đổi như sau:
- Năm 2021: Điểm chuẩn ngành Khoa học máy tính là 27.3 điểm.
- Năm 2022: Điểm chuẩn ngành Khoa học máy tính là 27.1 điểm.
Nếu bạn cần thêm thông tin hoặc có câu hỏi khác, đừng ngần ngại để lại cho mình biết nhé